# VALIDATION EXTERNE DES THEMES IDENTIFIES PAR LDA

## **OBJECTIF GENERAL**

Préparer la tâche de fouille de documents évaluée par un sondage auprès d'experts en SHS

## **OBJECTIFS SPECIFIQUES**

- Récupérer en ligne un champ lexical associé aux thèmes de recherche des rédacteurs des 3 revues d'étude : AE - RI - EI
- Mesurer (distance - mots_communs) la composition des thèmes trouvés par inférence avec la composition des thèmes a priori
- Mesurer la distance entre tous les mots d'un article identifié à un thème précis dominant et le champ lexical du thème extérieur

## **METHODE**

- Identifier les thèmes de recherche des rédacteurs des revues (cf *Erudit - Analyse.xls*)
- Rechercher sur https://www.rimessolides.com/motscles.aspx les champs lexicaux associés aux thèmes identifiés
- Récupérer les tokens (unigrammes et bigrammes) dans un dictionnaire du type {'theme_a_priori' : [tokens]}
- Filtrer avec les 30 mots les plus fréquents/sémantiquement liés dans ces tokens
- Compter le nombre de mots communs entre chaque thème LDA et les tokens des thèmes a priori
- En déduire un étiquettage de chaque thème si suffisamment de mots en commun (seuil ?)

- Construction d'une tâche d'évaluation : fouille de documents parmi chaque revue

**Import bibliothèques**

In [ ]:
from openpyxl import load_workbook

from gensim.models import LdaModel
from gensim import models, similarities
import gensim

import pandas as pd
import pandas as pdutifulSoup #parsing du XML
pd.set_option('display.max_colwidth', -1)

import urllib.request
import requests #requête des serveurs XML
from bs4 import BeautifulSoup


import collections
from tqdm import tqdm
import time
import random
import numpy as np
import matplotlib.pyplot as plt

## 1) IDENTIFICATION DES THEMES DE RECHERCHE DES REVUES

Output
- thèmes_AE
- thèmes_EI
- thèmes_RI

In [ ]:
cwd = 'C:/Users/arten/OneDrive - polymtl.ca/Documents/Etudes/Montréal/Etudes/Projet de recherche/Erudit/'

# Récupération du fichier excel
wb = load_workbook(cwd +'Erudit-Analyse.xlsx')

# Identification de la feuille d'intérêt
print(wb.sheetnames)
sheet = wb['Mots-clés']



**RELATIONS INDUSTRIELLES**

In [ ]:
# Récupération des thèmes sous forme de liste
thèmes_RI = sheet['A2'].value.split("\n")

**ETUDES INTERNATIONALES**

In [ ]:
# Récupération des thèmes sous forme de liste
thèmes_EI = sheet['B2'].value.split("\n")

**ACTUALITE ECONOMIQUE**

In [ ]:
# Récupération des thèmes sous forme de liste
thèmes_AE = sheet['C2'].value.split("\n")
thèmes_AE

**Résumé**

In [ ]:
print('Nombre de thèmes a priori identifiés pour les revues:\n', 'Actualité Economique :' , len(thèmes_AE), '\n Etudes internationales :' , len(thèmes_EI),'\n Relations Industrielles : ', len(thèmes_RI))

## 2) REQUETE ET CONSTITUTION DES CHAMPS LEXICAUX SUR RIMES SOLIDES

- Rechercher sur https://www.rimessolides.com/motscles.aspx les champs lexicaux associés aux thèmes identifiés
- Récupérer les tokens (unigrammes et bigrammes)

Output
- champ_lexical_AE
- champ_lexical_EI
- champ_lexical_RI

In [ ]:
def parse_url (liste_thèmes, url = 'https://www.rimessolides.com/motscles.aspx?m='):
    """Fonction qui parcoure la page web contenant le champ lexical de chaque thème et retourne une liste d'objets request"""
    liste_r = []
    for thème in liste_thèmes:
        try: 
            url_path = url + thème
            r = requests.get(url_path)
            liste_r.append((thème,r))
        except:
            print('Pas d\'url trouvé pour le thème ', thème)
    return liste_r

In [ ]:
def get_tokens(liste_r, thèmes):     
    """"Fonction qui parcoure la liste d'objets request et retourne un dictionnaire du type {thème : [liste tokens]}"""
    champ_lexical = {}
    for thème in thèmes:
        champ_lexical[thème] = []
    for thème, r in liste_r:
        soup = BeautifulSoup(r.content, "html.parser")
        for td in soup.findAll("table", class_ ="tb-motcle"):
            for element in td.findAll("a"):
                champ_lexical[thème].append(element.text)
    return champ_lexical

**Actualité économique**

In [ ]:
liste_url_AE = parse_url(thèmes_AE)

In [ ]:
champ_lexical_AE = get_tokens(liste_url_AE,thèmes_AE)

**Relations Industrielles**

In [ ]:
liste_url_RI = parse_url(thèmes_RI)

In [ ]:
champ_lexical_RI = get_tokens(liste_url_RI, thèmes_RI)

**Etudes Internationales**

In [ ]:
liste_url_EI = parse_url(thèmes_EI)

In [ ]:
champ_lexical_EI = get_tokens(liste_url_EI,thèmes_EI)

In [ ]:
%store champ_lexical_EI
%store champ_lexical_AE
%store champ_lexical_RI

## 3) MESURE DE SIMILARITE ENTRE THEME LDA ET THEME A PRIORI

- Préprocessing des mots issus de rimesolides pour permettre la mesure d'edit distance/mots communs avec les thèmes du LDA
- Comparaison entre les 2 pools de thèmes

Output:
- champ_lexical_EI_clean
- champ_lexical_AE_clean
- champ_lexical_RI_clean

In [ ]:
%store -r champ_lexical_EI
%store -r champ_lexical_AE
%store -r champ_lexical_RI

In [ ]:
champ_lexical_RI_clean.keys()

## Préprocessing des champs lexicaux

**Vérification du nombre de mots composant chaque thème (minimum 5 nécessaire)**

In [ ]:
longueur_EI = sorted([len(champ_lexical_EI[mot]) for mot in champ_lexical_EI])
longueur_AE = sorted([len(champ_lexical_AE[mot]) for mot in champ_lexical_AE])
longueur_RI = sorted([len(champ_lexical_RI[mot]) for mot in champ_lexical_RI])

**AE**

In [ ]:
plt.bar([i for i in range(len(longueur_AE))], longueur_AE)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')

In [ ]:
# Ajouter manuellement des thèmes plus larges
add_themes = ['Économétrie', 'Économie']
add_url_AE = parse_url(add_themes)
add_tokens = get_tokens(add_url_AE,add_themes)
champ_lexical_AE.update(add_tokens)

In [ ]:
# Enlever les thèmes avec trop peu de mots-clés (< 10 mots)
champ_lexical_AE = {i:champ_lexical_AE[i] for i in champ_lexical_AE if len(champ_lexical_AE[i]) > 10}

# Garder uniquement les 30 premiers mots-clés de chaque thème
restriction_tokens = {i:champ_lexical_AE[i][:30] for i in champ_lexical_AE if len(champ_lexical_AE[i]) > 30}

# Constitution du champ lexical final
champ_lexical_AE.update(restriction_tokens)

In [ ]:
champ_lexical_AE

**EI**

In [ ]:
plt.bar([i for i in range(len(longueur_EI))], longueur_EI)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')

In [ ]:
print([(item,len(value)) for (item,value) in champ_lexical_EI.items() if len(value) <10])

In [ ]:
champ_lexical_EI.keys()

In [ ]:
# Ajouter manuellement des thèmes plus larges
add_themes = ['Politique de défense', 'Stratégie']
add_url_EI = parse_url(add_themes)
add_tokens = get_tokens(add_url_EI,add_themes)
champ_lexical_EI.update(add_tokens)

In [ ]:
# Enlever les thèmes avec trop peu de mots-clés (< 10 mots)
champ_lexical_EI = {i:champ_lexical_EI[i] for i in champ_lexical_EI if len(champ_lexical_EI[i]) > 10}

# Garder uniquement les 30 premiers mots-clés de chaque thème
restriction_tokens = {i:champ_lexical_EI[i][:30] for i in champ_lexical_EI if len(champ_lexical_EI[i]) < 30}

# Constitution du champ lexical final
champ_lexical_EI.update(restriction_tokens)

In [ ]:
print([len(restriction_tokens[item]) for item in restriction_tokens])

In [ ]:
print([len(champ_lexical_EI[item]) for item in champ_lexical_EI])

In [ ]:
print(champ_lexical_EI)

**RI**

In [ ]:
plt.bar([i for i in range(len(longueur_RI))], longueur_RI)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')

In [ ]:
print([(item,len(value)) for (item,value) in champ_lexical_RI.items() if len(value) <5])

In [ ]:
# Ajouter manuellement des thèmes plus larges
add_themes = ['Immigration', 'Emploi']
add_url_RI = parse_url(add_themes)
add_tokens = get_tokens(add_url_RI,add_themes)
champ_lexical_RI.update(add_tokens)

In [ ]:
# Enlever les thèmes avec trop peu de mots-clés (< 10 mots)
champ_lexical_RI = {i:champ_lexical_RI[i] for i in champ_lexical_RI if len(champ_lexical_RI[i]) > 10}

# Garder uniquement les 30 premiers mots-clés de chaque thème
restriction_tokens = {i:champ_lexical_RI[i][:30] for i in champ_lexical_RI if len(champ_lexical_RI[i]) < 30}

# Constitution du champ lexical final
champ_lexical_RI.update(restriction_tokens)

**Vérification du nombre de mots composant chaque thème après filtre**

In [ ]:
longueur_EI = sorted([len(champ_lexical_EI[mot]) for mot in champ_lexical_EI])
longueur_AE = sorted([len(champ_lexical_AE[mot]) for mot in champ_lexical_AE])
longueur_RI = sorted([len(champ_lexical_RI[mot]) for mot in champ_lexical_RI])

In [ ]:
plt.figure(figsize=(15,5))

plt.bar([i for i in range(len(longueur_EI))], longueur_EI)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')

plt.xlabel('Thèmes')
plt.ylabel('Nombre de mots')
plt.title('Répartition du nombre de mots-clés pour chaque thème a priori pour la revue EI')
plt.savefig('Plots/Validité_Wikipedia/repartition_champ_lexical_EI')

In [ ]:
plt.figure(figsize=(15,5))
plt.bar([i for i in range(len(longueur_AE))], longueur_AE)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')
plt.xlabel('Thèmes')
plt.ylabel('Nombre de mots')
plt.title('Répartition du nombre de mots-clés pour chaque thème a priori pour la revue AE')
plt.savefig('Plots/Validité_Wikipedia/repartition_champ_lexical_AE')

In [ ]:
plt.figure(figsize=(15,5))
plt.bar([i for i in range(len(longueur_RI))], longueur_RI)
plt.axhline(y=5, color = 'red')
plt.axhline(y=30, color = 'purple')
plt.xlabel('Thèmes')
plt.ylabel('Nombre de mots')
plt.title('Répartition du nombre de mots-clés pour chaque thème a priori pour la revue RI')
plt.savefig('Plots/Validité_Wikipedia/repartition_champ_lexical_RI')

**Préparation des bigrammes**

In [ ]:
# mise en minuscule et constitution des bigrammes
def preprocess(champ_lexical):
    champ_lexical_clean = {}
    for index, tokens in champ_lexical.items():
        champ_lexical_clean[index] = [token.replace(' ', '_').lower() for token in tokens]
    return champ_lexical_clean

In [ ]:
champ_lexical_AE_clean = preprocess(champ_lexical_AE)
champ_lexical_EI_clean = preprocess(champ_lexical_EI)
champ_lexical_RI_clean = preprocess(champ_lexical_RI)

In [ ]:
%store champ_lexical_EI_clean
%store champ_lexical_AE_clean
%store champ_lexical_RI_clean

## Mesure de similarité

**Récupération des thèmes a priori**

In [ ]:
%store -r champ_lexical_EI_clean
%store -r champ_lexical_AE_clean
%store -r champ_lexical_RI_clean

**Récupération des thèmes des modèles LDA**

In [ ]:
def champ_lexical_LDA (model):
    """Retourner un dictionnaire pour un modèle lda donné sur une revue donnée de la forme 
        {num_topic : [liste tokens]}"""
    dico_mots = {}
    for theme in range(model.num_topics):
        topics = model.show_topic(theme, 10)
        dico_mots[theme] = [topics[i][0] for i in range(len(topics))]    
    return dico_mots

In [ ]:
def levenshteinDistance(s1, s2):
    """Mesure de string distance entre 2 string """
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [ ]:
def similarités_distance (champ_lexical_LDA, champ_lexical_clean):
    """Retourne les paires de thèmes (LDA, apriori) qui ont au moins un mot en commun ou deux mots à distance 1 l'un de l'autre"""
    compte_similarités = {}
    mots_proches = []
    for num_topic,mots_LDA in champ_lexical_LDA.items():
            for theme,mots_apriori in champ_lexical_clean.items():
                for mot_1 in mots_LDA:
                    for mot_2 in mots_apriori:
                        if levenshteinDistance(mot_1,mot_2) <= 1:
                            try : compte_similarités[(num_topic, theme)].append(mot_1)
                            except : compte_similarités[(num_topic, theme)] = [mot_1] 
    return compte_similarités 

In [ ]:
def attribution_topiques (model, compte_similarités):
    """"Retourne une étiquette à chaque thème avec les mots en commun"""
    thèmes_LDA = dict.fromkeys([x for x in range(model.num_topics)], [])
    for element in compte_similarités:
        if thèmes_LDA[element[0]] == []:
            thèmes_LDA[element[0]] = [(element[1], compte_similarités[element])]
        else : 
            thèmes_LDA[element[0]].append((element[1],compte_similarités[element]))
    return thèmes_LDA

In [ ]:
path_model = 'C:/Users/arten/OneDrive - polymtl.ca/Documents/Etudes/Montréal/Etudes/Projet de recherche/Code/Erudit/Résultats_LDA'

In [ ]:
# Choix du nombre de thèmes pour les modèles LDA
num_topics = [x for x in range(10,110,10)]
num_topics

In [ ]:
def computing_thèmes (champ_lexical_clean, revue = '/lda_ae_'):
    """Fonction pour itérer sur les différents modèles de thèmes pour une revue"""
    thèmes_revue = dict.fromkeys([x for x in num_topics])
    for i in num_topics:
        print(path_model + revue + str(i))
        model = LdaModel.load(path_model + revue + str(i))
        champ_lexical = champ_lexical_LDA(model)
        compte_similarités = similarités_distance(champ_lexical, champ_lexical_clean)
        thèmes_revue[i] = attribution_topiques(model,compte_similarités)
    return thèmes_revue

**AE**

In [ ]:
thèmes_revue_AE = computing_thèmes(champ_lexical_AE_clean, revue='/lda_ae_')

In [ ]:
%store thèmes_revue_AE

**EI**

In [ ]:
thèmes_revue_EI = computing_thèmes(champ_lexical_EI_clean, revue='/lda_ei_')

In [ ]:
%store thèmes_revue_EI

**RI**

In [ ]:
thèmes_revue_RI = computing_thèmes(champ_lexical_RI_clean, revue='/lda_ri_')

In [ ]:
%store thèmes_revue_RI

## 4) VALIDATION DU MODELE SUR UNE TACHE DE FOUILLE DE DOCUMENT

**OBJECTIF**
- Valider l'information thématique extraite avec le modèle LDA sur une tâche de fouille de documents

**METHODE**
- 1ère étape : Identifier les correspondances des thèmes entre LDA et a priori (descriptif)
- 2ème étape : Identifier les documents dans la revue contenant les thèmes LDA majoritaires parmi ceux filtrés en 1ère étape
- 3ème étape : Identifier les mots-clés (Sélection fréquentielle) contenus dans ces documents (constitution de (a))
- 4ème étape : Etude quantitative des 4 méthodes de fouille : 
    - a) Fouille de documents à l'aide des mots-clés issus des documents majoritaires
    - b) Fouille de documents à l'aide des mots-clés issus des thèmes a priori
    - c) Fouille de documents à l'aide des mots-clés issus des thèmes LDA
    - d) Fouille de documents à l'aide des distances dans l'espace latent lda (avec 3 modèles de thèmes)

### 1) Etude comparative thèmes LDA et thèmes a priori

**Calcul du nombre de thèmes LDA identifiés avec un thème a priori**

In [ ]:
%store -r thèmes_revue_RI
%store -r thèmes_revue_EI
%store -r thèmes_revue_AE

In [ ]:
def compte_cohérence(thèmes_revue):
    """Compte le nombre de thèmes LDA auquel au moins un thème a priori a pu être identifié"""
    compte = []
    for num_thème, dico_thèmes in thèmes_revue.items():
        compteur = 0
        for _ , thème in dico_thèmes.items():
            if thème == []:
                compteur +=1
        compte.append((num_thème,compteur))
    return compte

In [ ]:
compte_cohérence_AE = compte_cohérence(thèmes_revue_AE)
compte_cohérence_EI = compte_cohérence(thèmes_revue_EI)
compte_cohérence_RI = compte_cohérence(thèmes_revue_RI)

**Plots**

In [ ]:
path_save = "Plots/Validité_Wikipedia/"
# Enregistré sur le One drive directement

In [ ]:
x = [element[0] for element in compte_cohérence_AE]
y = [int(element[1]/element[0]*100) for element in compte_cohérence_AE]
plt.figure(figsize=(10,10))
plt.title("Pourcentage de thèmes incohérents en fonction de la capacité du modèle LDA pour la revue AE")
plt.xlabel("Nombre de thèmes du modèle LDA")
plt.ylabel("Pourcentage de thèmes incohérents")
plt.axhline(y = np.mean(y), color = 'orange')
plt.bar(x,y)
plt.savefig(path_save + "pourcentage_incohérence_AE.png")

In [ ]:
x = [element[0] for element in compte_cohérence_EI]
y = [int(element[1]/element[0]*100) for element in compte_cohérence_EI]
plt.figure(figsize=(10,10))
plt.title("Pourcentage de thèmes incohérents en fonction de la capacité du modèle LDA pour la revue EI")
plt.xlabel("Nombre de thèmes du modèle LDA")
plt.ylabel("Pourcentage de thèmes incohérents")
plt.axhline(y = np.mean(y), color = 'orange')
plt.bar(x,y)
plt.savefig(path_save + "pourcentage_incohérence_EI.png")

In [ ]:
x = [element[0] for element in compte_cohérence_RI]
y = [int(element[1]/element[0]*100) for element in compte_cohérence_RI]
plt.figure(figsize=(10,10))
plt.title("Pourcentage de thèmes incohérents en fonction de la capacité du modèle LDA pour la revue RI")
plt.xlabel("Nombre de thèmes du modèle LDA")
plt.ylabel("Pourcentage de thèmes incohérents")
plt.axhline(y = np.mean(y), color = 'orange')
plt.bar(x,y)
plt.savefig(path_save + "pourcentage_incohérence_RI.png")

**Filtre des thèmes LDA identifiés comme "cohérents" selon les thèmes a priori**

In [ ]:
def thèmes_revues_cohérents(thèmes_revue):
    """Retourne les thèmes LDA auquel au moins un thème a priori a pu être identifié"""
    thèmes_cohérents = dict.fromkeys([x for x in num_topics])
    for num_thème, dico_thèmes in thèmes_revue.items():
        thèmes ={}
        for index , thème in dico_thèmes.items():
            if thème != []:
                thèmes[index] = thème
        thèmes_cohérents[num_thème] = thèmes
    return thèmes_cohérents

In [ ]:
thèmes_revue_cohérents_AE = thèmes_revues_cohérents(thèmes_revue_AE)
thèmes_revue_cohérents_EI = thèmes_revues_cohérents(thèmes_revue_EI)
thèmes_revue_cohérents_RI = thèmes_revues_cohérents(thèmes_revue_RI)

In [ ]:
%store thèmes_revue_cohérents_AE
%store thèmes_revue_cohérents_RI
%store thèmes_revue_cohérents_EI

### 2) Identification des articles contenant la plus grande proportion de thèmes et récupération des tokens de ces articles

In [ ]:
%store -r thèmes_revue_cohérents_AE
%store -r thèmes_revue_cohérents_RI
%store -r thèmes_revue_cohérents_EI

In [ ]:
%store -r corpus_train_AE
%store -r corpus_train_EI
%store -r corpus_train_RI

In [ ]:
def get_topics_document(ldamodel, corpus, texts):
    """Retourne une liste où chaque élément représente pour chaque article [index_thème_dominant, prop_thème, mots_clés_thème, tokens_article]"""
    sentences = []
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # On récupère le thème majoritaire, sa contribution en % et les mots-clés du thème et les mots-clés du thème pour chaque document ainsi que les tokens du document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => on s'intéresse au thème dominant
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sentences.append([int(topic_num), round(prop_topic,4), topic_keywords,texts[i]])
            else:
                break
    return(sentences)

In [ ]:
def filtre_thèmes(sorted_sentences, thèmes_cohérents, ldamodel):
    """Retourner pour chaque thème LDA cohérent les 5 documents qui contiennent le plus de ce thème en proportion
     Output = dictionnaire avec clé = index_thème et valeur = liste des tokens des 5 articles ayant ces thèmes en plus grande contribution"""
    filtres = {}
    for index_thème, autre in thèmes_cohérents[ldamodel.num_topics].items():
        filtre = []
        for element in sorted_sentences:
            if element[0] == index_thème and len(filtre)<5 :
                filtre.append(element[3])
        filtres[index_thème] = filtre
    return filtres

**Récupération des modèles**

In [ ]:
%store -r train_texts_AE
%store -r train_texts_EI
%store -r train_texts_RI

In [ ]:
path_model = 'C:/Users/arten/OneDrive - polymtl.ca/Documents/Etudes/Montréal/Etudes/Projet de recherche/Code/Erudit/Résultats_LDA'
i = 10 # nombre de thèmes du modèle

In [ ]:
# AE
revue = '/lda_ae_'
model_ae_10 = LdaModel.load(path_model + revue + str(i))

In [ ]:
# EI
revue = '/lda_ei_'
model_ei_10 = LdaModel.load(path_model + revue + str(i))

In [ ]:
# AE
revue = '/lda_ri_'
model_ri_10 = LdaModel.load(path_model + revue + str(i))

In [ ]:
sentences_AE = get_topics_document(ldamodel=model_ae_10, corpus=corpus_train_AE, texts=train_texts_AE)
sentences_EI = get_topics_document(ldamodel=model_ei_10, corpus=corpus_train_EI, texts=train_texts_EI)
sentences_RI = get_topics_document(ldamodel=model_ri_10, corpus=corpus_train_RI, texts=train_texts_RI)

In [ ]:
# Trier la liste selon les thèmes dominants et les proportion de ces thèmes au sein des articles
sorted_sentences_AE = sorted(sentences_AE, key =lambda tup:(tup[0],tup[1]), reverse=True) 
sorted_sentences_EI = sorted(sentences_EI, key =lambda tup:(tup[0],tup[1]), reverse=True) 
sorted_sentences_RI = sorted(sentences_RI, key =lambda tup:(tup[0],tup[1]), reverse=True) 

In [ ]:
sorted_sentences_EI

In [ ]:
filtre_thèmes_AE = filtre_thèmes(sorted_sentences = sorted_sentences_AE, thèmes_cohérents = thèmes_revue_cohérents_AE, ldamodel = model_ae_10)
filtre_thèmes_EI = filtre_thèmes(sorted_sentences = sorted_sentences_EI, thèmes_cohérents = thèmes_revue_cohérents_EI, ldamodel = model_ei_10)
filtre_thèmes_RI = filtre_thèmes(sorted_sentences = sorted_sentences_RI, thèmes_cohérents = thèmes_revue_cohérents_RI, ldamodel = model_ri_10)

In [ ]:
%store filtre_thèmes_AE 
%store filtre_thèmes_EI 
%store filtre_thèmes_RI

### 3) Sélection fréquentielle des tokens présents dans les 5 articles majoritaires

In [ ]:
%store -r filtre_thèmes_AE 
%store -r filtre_thèmes_EI 
%store -r filtre_thèmes_RI

In [ ]:
def filtre_tokens(filtre_thèmes, diversité = 10):
    """Constituer un vocabulaire de mots-clés associés à chaque thème à partir des articles où ces thèmes sont les plus importants
    Paramètres : 
    - diversité = nombre de tokens différents récupérés dans chaque article
    le vocabulaire total sera de 5 * diversité"""
    
    vocabulaire_thèmes = {}
    for theme, liste_article in filtre_thèmes.items():
        voca = []
        for article in liste_article:
            count = collections.Counter(article)
            count = count.most_common(diversité)
            for element in count:
                    voca.append(element[0])
        vocabulaire_thèmes[theme] = voca
    return vocabulaire_thèmes

In [ ]:
voca_AE_docmaj = filtre_tokens(filtre_thèmes_AE, diversité = 10)
voca_EI_docmaj = filtre_tokens(filtre_thèmes_EI, diversité = 10)
voca_RI_docmaj = filtre_tokens(filtre_thèmes_RI, diversité = 10)

In [ ]:
%store voca_AE_docmaj
%store voca_EI_docmaj
%store voca_RI_docmaj

### 4) Tâche de fouille de documents

#### i. Préparation des listes de mots-clés

Constitution et récupération des 4 listes de mots-clés pour la recherche :
- Liste issues des tokens des articles majoritaires de chaque thème = **voca_AE_docmaj - voca_EI_docmaj - voca_RI_docmaj**
- Liste issues des thèmes a priori dont on a trouvé une similarité avec thèmes LDA : **voca_AE_apriori - voca_EI_apriori - voca_RI_apriori**
- Liste issues des tokens de chaque thème LDA : **voca_AE_LDA - voca_EI_LDA - voca_RI_LDA**
- Liste de tokens aléatoires issus du vocabulaire de l'ensemble de la revue : **voca_AE_random - voca_EI_random - voca_RI_random**

**a) Voca issu des thèmes des documents majoritaires**

In [ ]:
# Vocabulaire de mots-clés (50 par thème) issus des 5 documents où les thèmes LDA étaient les plus présents
%store -r voca_AE_docmaj
%store -r voca_EI_docmaj
%store -r voca_RI_docmaj

**b) Voca issu des thèmes a priori**

In [ ]:
# Champ lexical de chaque thème a priori sous forme de dictionnaire
%store -r champ_lexical_EI_clean
%store -r champ_lexical_AE_clean
%store -r champ_lexical_RI_clean

In [ ]:
# Correspondances entre thèmes LDA et thèmes a priori
%store -r thèmes_revue_cohérents_AE
%store -r thèmes_revue_cohérents_RI
%store -r thèmes_revue_cohérents_EI

In [ ]:
# On ne garde que les thèmes a priori dont on a trouvé une correspondance avec un thème LDA
def voca_apriori(thèmes_revue_cohérents, champ_lexical_clean, num_topics=10):
    """Retourne un dictionnaire avec chaque élément du type [index_thème_LDA, liste de tokens des thèmes a priori associés]"""
    themes_apriori = {}
    for thème_LDA, element in thèmes_revue_cohérents[num_topics].items():      
        themes_apriori[thème_LDA] = [element[i][0] for i in range(len(element))]
    
    print(themes_apriori)
    voca_apriori = {}
    for index, theme in themes_apriori.items():
        try:
            voca_apriori[index] = champ_lexical_clean[theme]
        except: 
            voca_apriori[index] = [champ_lexical_clean[theme[i]] for i in range(len(theme))]
        voca_apriori[index] = [element for sublist in voca_apriori[index] for element in sublist] # flatten list of list
    
    return voca_apriori

In [ ]:
voca_AE_apriori = voca_apriori(thèmes_revue_cohérents_AE, champ_lexical_AE_clean, num_topics=10)
voca_EI_apriori = voca_apriori(thèmes_revue_cohérents_EI, champ_lexical_EI_clean, num_topics=10)
voca_RI_apriori = voca_apriori(thèmes_revue_cohérents_RI, champ_lexical_RI_clean, num_topics=10)

In [ ]:
#Mots-clés des thèmes a priori
%store voca_AE_apriori
%store voca_EI_apriori
%store voca_RI_apriori

**c) Voca issu des thèmes LDA**

In [ ]:
def voca_LDA(ldamodel):
    """Retourne les tokens de chaque thème"""
    voca = {}
    for theme in range(ldamodel.num_topics):
        voca[theme] = [mot for mot, _ in ldamodel.show_topic(theme)]
    return(voca)

In [ ]:
voca_AE_LDA = voca_LDA(model_ae_10)
voca_EI_LDA = voca_LDA(model_ei_10)
voca_RI_LDA = voca_LDA(model_ri_10)

In [ ]:
%store voca_AE_LDA
%store voca_EI_LDA
%store voca_RI_LDA

**d) Voca issus de thèmes random**

In [ ]:
%store -r dictionary_AE_2 
%store -r dictionary_EI_2 
%store -r dictionary_RI_2 

In [ ]:
def voca_aléatoire(dictionary):
    voca = [mots for mots in dictionary_AE_2.values()]
    random.shuffle(voca)
    return voca

In [ ]:
voca_AE_aléatoire = voca_aléatoire(dictionary_AE_2)
voca_EI_aléatoire = voca_aléatoire(dictionary_EI_2)
voca_RI_aléatoire = voca_aléatoire(dictionary_RI_2)

In [ ]:
%store voca_AE_aléatoire
%store voca_EI_aléatoire
%store voca_RI_aléatoire

#### ii. Requête et évaluation

#### BUT :  Trouver les documents du corpus les plus pertinents sur un thème d'intérêt a priori

#### METHODE

*Répéter pour chaque revue et chaque thème apriori*

A) Choisir un thème a priori et identifier les thèmes LDA associés (à l'aide de thèmes_revue_cohérents)

B) Constituer le corpus sur lequel on applique la requête

C) Fouiller le contenu des documents à l'aide des mots-clés associés à :
- 10 mots-clés du thème a priori
- 10 mots-clés des thèmes LDA associé à ce thème apriori
- 10 mots-clés issus des documents où les thèmes LDA sont majoritaires
- 10 mots-clés aléatoire du vocabulaire total

Retourner les 5 documents du corpus les plus pertinents 

**1ère mesure de pertinence = Compter le nombre d'occurences des mots-clés dans leur contenu pour chacune des méthode de fouille**

**2ème mesure de pertinence = Mesure de similarité cosinus entre le vecteur de requête et les vecteurs des documents au sein de la représentation LDA**

D) Analyse visuelle de pertinence

# **Méthode 1 : Mesure en fréquence**

In [ ]:
# Récupération des 4 listes de vocabulaire (pour 10 topics)
%store -r voca_AE_docmaj
%store -r voca_EI_docmaj
%store -r voca_RI_docmaj

%store -r voca_AE_apriori
%store -r voca_EI_apriori
%store -r voca_RI_apriori

%store -r voca_AE_LDA
%store -r voca_EI_LDA
%store -r voca_RI_LDA

%store -r voca_AE_aléatoire
%store -r voca_EI_aléatoire
%store -r voca_RI_aléatoire

# Correspondances entre thèmes LDA et thèmes a priori
%store -r thèmes_revue_cohérents_AE
%store -r thèmes_revue_cohérents_RI
%store -r thèmes_revue_cohérents_EI

In [ ]:
%store -r tokens_bigrams_Corpus_LDA_AE_clean
%store -r tokens_bigrams_Corpus_LDA_EI_clean
%store -r tokens_bigrams_Corpus_LDA_RI_clean

In [ ]:
%store -r Corpus_AE
%store -r Corpus_EI
%store -r Corpus_RI

A) Constituer le corpus de requête **(TOURNER 1 FOIS SEULEMENT)**

In [ ]:
def get_url_metadata (Corpus, tokens):
    """Prend en paramètre le Corpus avec textes et métadonnées et retourne le Corpus filtré avec les articles en français et de type article
    + les tokens nettoyés pour chacun d'entre eux et l'URL"""
    Corpus_filtre = []
    index_clean = 0
    for index_document in tqdm(range(len(Corpus))) :
    # on ne traite que les articles de type article et en français
        if (Corpus[index_document]['metadata']['typeart'], Corpus[index_document]['metadata']['lang']) == ('article','fr') : 
            Corpus_filtre.append((Corpus[index_document]['URL'], tokens[index_clean]))
            index_clean +=1
    return Corpus_filtre

In [ ]:
Corpus_fouille_AE = get_url_metadata(Corpus_AE, tokens_bigrams_Corpus_LDA_AE_clean)
Corpus_fouille_EI = get_url_metadata(Corpus_EI, tokens_bigrams_Corpus_LDA_EI_clean)
Corpus_fouille_RI = get_url_metadata(Corpus_RI, tokens_bigrams_Corpus_LDA_RI_clean)

In [ ]:
%store Corpus_fouille_AE
%store Corpus_fouille_EI
%store Corpus_fouille_RI

# COTE UTILISATEUR

B) Retourner pour un thème a priori le numéro thèmes LDA candidats (parcours dico inverse)

In [ ]:
def get_topics_lda(thèmes_revue_cohérents,theme = 'Économie', num_topics=10):
    """Retourne la liste des numéros des thèmes LDA dont le thème a priori partage un ou plusieurs mots-clés"""
    themes_lda = []
    for theme_LDA, element in thèmes_revue_cohérents[num_topics].items():
        for i in range(len(element)):
            if element[i][0] == theme: themes_lda.append(theme_LDA)
    return themes_lda

C) Fouille à l'aide des 4 méthodes

In [ ]:
%store -r Corpus_fouille_AE
%store -r Corpus_fouille_EI
%store -r Corpus_fouille_RI

In [ ]:
%store -r tokens_bigrams_Corpus_LDA_AE_clean
%store -r tokens_bigrams_Corpus_LDA_EI_clean
%store -r tokens_bigrams_Corpus_LDA_RI_clean

texts_AE = tokens_bigrams_Corpus_LDA_AE_clean
texts_EI = tokens_bigrams_Corpus_LDA_EI_clean
texts_RI = tokens_bigrams_Corpus_LDA_RI_clean

In [ ]:
# Récupération des 4 listes de vocabulaire (pour 10 topics)
%store -r voca_AE_docmaj
%store -r voca_EI_docmaj
%store -r voca_RI_docmaj

%store -r voca_AE_apriori
%store -r voca_EI_apriori
%store -r voca_RI_apriori

%store -r voca_AE_LDA
%store -r voca_EI_LDA
%store -r voca_RI_LDA

%store -r voca_AE_aléatoire
%store -r voca_EI_aléatoire
%store -r voca_RI_aléatoire

# Correspondances entre thèmes LDA et thèmes a priori
%store -r thèmes_revue_cohérents_AE
%store -r thèmes_revue_cohérents_RI
%store -r thèmes_revue_cohérents_EI

In [ ]:
champ_lexical_AE.keys()

In [ ]:
# Thèmes a priori
%store -r champ_lexical_AE
%store -r champ_lexical_EI
%store -r champ_lexical_RI

In [ ]:
def search_corpus(mots_clés, texts):
    """Retourne un dictionnaire avec index des articles où se trouvent les mots clés + compteur d'occurrences des mots-clés"""
    documents_found = {}
    for text in texts:
        documents_found[texts.index(text)] = 0
        for mot in mots_clés:
            if text.count(mot) != 0 : documents_found[texts.index(text)] += text.count(mot)
    return documents_found

In [ ]:
def fouille_corpus(themes_lda, corpus_fouille = Corpus_fouille_AE, texts= texts_AE, voca_LDA = voca_AE_LDA, voca_docmaj =voca_AE_docmaj, voca_apriori = voca_AE_apriori, voca_aléatoire = voca_AE_aléatoire):
    """Input : un intérêt de recherche qui nous sert de clé de recherche dans le corpus
        + 4 vocabulaires construits à l'aide de méthodes différentes : thèmes LDA - thèmes des documents où le thème est majoritaire - thèmes a_priori - mots aléatoires
        Output : Pour chaque méthode, sortir 5 documents plus pertinents pour l'intérêt de recherche"""
    
    # Modifier l'index (pour idnex=0 : on ne prend qu'un seul thème LDA proche des thèmes a priori)
    index_lda = themes_lda[0]
    
    documents = {}
    # Mots-clés issus des thèmes LDA
    requete = search_corpus(voca_LDA[index_lda], texts)
    documents['LDA'] = sorted(requete.items(), key=lambda x:x[1], reverse = True)[:5] # 5 premiers documents
    
    # Mots-clés issus des documents majoritair
    requete = search_corpus(voca_docmaj[index_lda], texts)
    documents['docmaj'] = sorted(requete.items(), key=lambda x:x[1], reverse = True)[:5]
    
    # Mots-clés issus des thèmes a priori
    requete = search_corpus(voca_apriori[index_lda], texts)
    documents['apriori'] = sorted(requete.items(), key=lambda x:x[1], reverse = True)[:5]
    
    # Mots-clés issus des documents majoritaires
    
    requete = search_corpus(voca_aléatoire[0:10], texts)
    documents['aléatoire'] = sorted(requete.items(), key=lambda x:x[1], reverse = True)[:5]
    
    URLS = {}
    for voca, liste in documents.items():
        urls = []
        for i in range(5):
            url = documents[voca][i][0]
            urls.append(corpus_fouille[url][0])
        URLS[voca] = urls
    
    return documents, URLS

In [ ]:
def get_title_from_URL (URL):
    r = requests.get(URL + '/ERUDITXSD300.xml', auth=('mgagnon', 'iathe6eiyaiy6Eic')) 
    soup = BeautifulSoup(r.text, "lxml")
    return soup.titre.text

In [ ]:
def requête_lda_frequence(revue = 'ae', theme = None, num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_AE, 
                          texts= texts_AE, voca_LDA = voca_AE_LDA, voca_docmaj =voca_AE_docmaj, voca_apriori = voca_AE_apriori, voca_aléatoire = voca_AE_aléatoire, print_=True, save=True):
    t0 = time.time()
    if theme == None:
        theme = input("Veuillez entrer votre requête ici :")
    
    themes_lda = get_topics_lda(theme = theme, num_topics=num_topics, thèmes_revue_cohérents=thèmes_revue_cohérents)
    requete, URLS = fouille_corpus(themes_lda = themes_lda, texts= texts, voca_LDA = voca_LDA, voca_docmaj =voca_docmaj, voca_apriori = voca_apriori, voca_aléatoire = voca_aléatoire)
    
    print("Fin de la conversion de la requête en ", round(time.time()-t0,0), "secondes")
    # Récupération des titres des articles
    data = dict.fromkeys(URLS.keys())
    for key, value in tqdm(URLS.items()):
        for URL in value:
            try :
                data[key].append(get_title_from_URL(URL))
            except : 
                data[key] = [get_title_from_URL(URL)]
    if print_:
        print(pd.DataFrame(data))
    
    if save: 
        pd.DataFrame(data).to_csv("Plots/LDA/fouille/"+revue +"/fouille_frequence_" + revue +"_query="+theme+"_num_topics=" + str(num_topics) +".csv")
        
    return pd.DataFrame(data)

In [ ]:
themes_lda = get_topics_lda(theme = 'Économétrie', num_topics=10, thèmes_revue_cohérents=thèmes_revue_cohérents_AE)

In [ ]:
themes_lda

In [ ]:
print(voca_AE_LDA[themes_lda[0]],'\n \n',voca_AE_docmaj[themes_lda[0]], '\n \n', voca_AE_apriori[themes_lda[0]], '\n \n', voca_AE_aléatoire[:10])

In [ ]:
resultat_AE = requête_lda_frequence(revue = 'ae', num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_AE, 
                                  texts= texts_AE, voca_LDA = voca_AE_LDA, voca_docmaj =voca_AE_docmaj, voca_apriori = voca_AE_apriori, voca_aléatoire = voca_AE_aléatoire, print_=True)

In [ ]:
for theme in champ_lexical_EI.keys():
    themes_lda = get_topics_lda(theme = theme, num_topics=10, thèmes_revue_cohérents=thèmes_revue_cohérents_EI)
    print(themes_lda,theme)

In [ ]:
resultat_EI = requête_lda_frequence(revue = 'ei', num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_EI, 
                                  texts= texts_EI, voca_LDA = voca_EI_LDA, voca_docmaj =voca_EI_docmaj, voca_apriori = voca_EI_apriori, voca_aléatoire = voca_EI_aléatoire, print_=True)

In [ ]:
resultat_EI = requête_lda_frequence(revue = 'ei', num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_EI, 
                                  texts= texts_EI, voca_LDA = voca_EI_LDA, voca_docmaj =voca_EI_docmaj, voca_apriori = voca_EI_apriori, voca_aléatoire = voca_EI_aléatoire, print_=True)

In [ ]:
for theme in champ_lexical_RI.keys():
    themes_lda = get_topics_lda(theme = theme, num_topics=10, thèmes_revue_cohérents=thèmes_revue_cohérents_RI)
    print(themes_lda,theme)

In [ ]:
resultat_RI = requête_lda_frequence(revue = 'ri', num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_RI, 
                                  texts= texts_RI, voca_LDA = voca_RI_LDA, voca_docmaj =voca_RI_docmaj, voca_apriori = voca_RI_apriori, voca_aléatoire = voca_RI_aléatoire, print_=True)

In [ ]:
resultat_RI = requête_lda_frequence(revue = 'ri', num_topics=10,thèmes_revue_cohérents=thèmes_revue_cohérents_RI, 
                                  texts= texts_RI, voca_LDA = voca_RI_LDA, voca_docmaj =voca_RI_docmaj, voca_apriori = voca_RI_apriori, voca_aléatoire = voca_RI_aléatoire, print_=True)

**Influence du nombre de thèmes**

Il faut créer un vocabulaire pour chaque thème spécifique

In [ ]:
num_topics = [x for x in range(10,110,10)]
num_topics

In [ ]:
def get_themes_for_all_topics (theme = 'Économie', thèmes_revue =thèmes_revue_cohérents_AE):
    """Retourne pour chaque topic model l'index des thèmes LDA qui ont au moins un mot en commun avec le thème recherché"""
    themes = dict.fromkeys(num_topics)
    for topic in tqdm(num_topics):
        themes[topic] = get_topics_lda(theme = theme, num_topics=topic, thèmes_revue_cohérents= thèmes_revue)
    return themes

In [ ]:
themes_AE = get_themes_for_all_topics(theme = 'Économie', thèmes_revue =thèmes_revue_cohérents_AE)

In [ ]:
themes_AE

# **Méthode 2 : Mesure de similarité cosinus dans l'espace LDA**

-------------------------

## TOURNER UNE SEULE FOIS : création des index pour chaque modèle

In [ ]:
# Indexation des documents du corpus pour fouiller plus rapidement
%store -r dictionary_AE_2
dictionary = dictionary_AE_2
revue = '/lda_ae_'
corpus_lda_AE = [dictionary.doc2bow(doc) for doc in tokens_bigrams_Corpus_LDA_AE_clean]

for topic in tqdm(range(10,110,10)):
    model = models.LdaModel.load('Résultats_LDA'+ revue + str(topic))
    index = similarities.MatrixSimilarity(model[corpus_lda_AE])
    index.save('Variables résultats/index/index_ae_' + str(topic))

In [ ]:
# Modèles fiabilité
%store -r dictionary_AE_2
dictionary = dictionary_AE_2
revue = '/lda_ae_'
corpus_lda_AE = [dictionary.doc2bow(doc) for doc in tokens_bigrams_Corpus_LDA_AE_clean]

for topic in tqdm(range(5)):
    model = models.LdaModel.load('Résultats_LDA/Fiabilité'+ revue + str(10) +"_"+str(topic))
    index = similarities.MatrixSimilarity(model[corpus_lda_AE])
    index.save('Variables résultats/index/index_ae_' + str(topic))

In [ ]:
# Indexation des documents du corpus pour fouiller plus rapidement
%store -r dictionary_EI_2
dictionary = dictionary_EI_2
revue = '/lda_ei_'
corpus_lda = [dictionary.doc2bow(doc) for doc in tokens_bigrams_Corpus_LDA_EI_clean]

for topic in tqdm(range(10,110,10)):
    model = models.LdaModel.load('Résultats_LDA'+ revue + str(topic))
    index = similarities.MatrixSimilarity(model[corpus_lda])
    index.save('Variables résultats/index/index_ei_' + str(topic))

In [ ]:
# Indexation des documents du corpus pour fouiller plus rapidement
%store -r dictionary_RI_2
dictionary = dictionary_RI_2
revue = '/lda_ri_'
corpus_lda = [dictionary.doc2bow(doc) for doc in tokens_bigrams_Corpus_LDA_RI_clean]

for topic in tqdm(range(10,110,10)):
    model = models.LdaModel.load('Résultats_LDA'+ revue + str(topic))
    index = similarities.MatrixSimilarity(model[corpus_lda])
    index.save('Variables résultats/index/index_ri_' + str(topic))

--------------------

## COTE UTILISATEUR

In [ ]:
%store -r dictionary_AE_2
%store -r dictionary_EI_2
%store -r dictionary_RI_2

In [ ]:
def print_title_from_URL (URL):
    r = requests.get(URL + '/ERUDITXSD300.xml', auth=('mgagnon', 'iathe6eiyaiy6Eic')) 
    soup = BeautifulSoup(r.text, "lxml")
    print(URL, ' : ', soup.titre.text)

In [ ]:
def get_title_and_URL (indexes, corpus=Corpus_fouille_AE):
    data = []
    for index in indexes:
        URL = corpus[index][0]
        r = requests.get(URL + '/ERUDITXSD300.xml', auth=('mgagnon', 'iathe6eiyaiy6Eic')) 
        soup = BeautifulSoup(r.text, "lxml")
        data.append(soup.titre.text)
    return data

In [ ]:
def requête_LDA(revue = 'ae', num_topics=0, corpus = Corpus_fouille_AE, dictionary= dictionary_AE_2, query = None, print_=True, get=True, fiabilité=False):
    """Fonction pour permettre la recherche de mots-clés dans le corpus à l'aide de la similarité cosinus"""
    
    if fiabilité==True:
        model = models.LdaModel.load('Résultats_LDA/Fiabilité/lda_'+ revue +'_'+ str(10) +"_"+str(topic))
        index = similarities.MatrixSimilarity.load('Variables résultats/index/index_' + revue + '_' + str(num_topics))
    else:
        model = models.LdaModel.load('Résultats_LDA/'+revue+ '/lda_'+ revue + '_'+str(num_topics))
        index = similarities.MatrixSimilarity.load('Variables résultats/index/index_' + revue + '_' + str(num_topics))
    
    if query == None:
        query = input('Veuillez entrer votre requête : ')

    # Transformation du string en matrice BoW
    vec_query = dictionary.doc2bow(query.lower().split())    
    
    # Requête placée dans l'espace du modèle LDA
    vec_lda = model[vec_query]

    # Récupération des documents les plus proches du vecteur de requête
    sims = index[vec_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1]) # ordonnement par ordre décroissant de similarité cosinus

    ##### Récupération des index des documents les plus pertinents et impression des URL associés
    indexes = [i[0] for i in sims[:5]]
    
    if print_:
        for index in indexes:
            print_title_from_URL(corpus[index][0])
    
    if get:
        resultat = get_title_and_URL(indexes,corpus)
        return resultat

In [ ]:
# Exemple
resultat_AE = requête_LDA(revue = 'ae', num_topics=10, corpus = Corpus_fouille_AE, dictionary = dictionary_AE_2, print_=True, get=True, fiabilité=False)

In [ ]:
# Exemple
resultat_AE = requête_LDA(revue = 'ae', query='Macroéconomie', num_topics=4, corpus = Corpus_fouille_AE, dictionary = dictionary_AE_2, print_=True, get=True, fiabilité=True)

In [ ]:
resultat_EI = requête_LDA(revue = 'ei', corpus = Corpus_fouille_EI, dictionary = dictionary_EI_2, num_topics=50, print_=True, get=True)

In [ ]:
resultat_RI = requête_LDA(revue = 'ri', corpus = Corpus_fouille_RI, dictionary = dictionary_RI_2, num_topics=50, print_=True, get=True)

**Etude de l'influence du nombre de thèmes sur le résultat de la requête**

In [ ]:
def influence_thèmes(query=None, save = True, print_=True, topic_max=30, revue='ae',corpus = Corpus_fouille_AE, dictionary = dictionary_AE_2):
    if query == None:
        query=input("Entrez vore requête ici : ")
    data={}
    for topic in tqdm(range(10, topic_max+10,10)):
        data[topic] = requête_LDA(revue = revue, corpus = corpus, dictionary = dictionary, num_topics=topic, query=query, get=True, print_=False)
        
    if save == True:
        pd.DataFrame(data).to_csv("Plots/LDA/fouille/"+ revue +"/fouille_" +revue +"_query="+query+".csv")
    
    if print_ == True :
        pd.DataFrame(data).head()
        
    return pd.DataFrame(data)

**AE**

In [ ]:
panda = influence_thèmes(save=True, print_=True, topic_max=50)

In [ ]:
panda.head()

In [ ]:
panda = influence_thèmes(query='politique monétaire',save=False, print_=True, topic_max=20)

In [ ]:
panda = influence_thèmes(query='Économie du travail',save=True, print_=True, topic_max=50)

In [ ]:
panda = influence_thèmes(query='Méthodes numériques',save=True, print_=True, topic_max=50)

**EI**

In [ ]:
panda = influence_thèmes(save=True, print_=True, topic_max=50, revue='ei',corpus = Corpus_fouille_EI, dictionary = dictionary_EI_2)

In [ ]:
panda.head()

In [ ]:
panda = influence_thèmes(query='Sécurité internationale',save=True, print_=True, topic_max=50, revue='ei',corpus = Corpus_fouille_EI, dictionary = dictionary_EI_2)

In [ ]:
panda = influence_thèmes(query='afrique',save=True, print_=True, topic_max=50, revue='ei',corpus = Corpus_fouille_EI, dictionary = dictionary_EI_2)

In [ ]:
panda = influence_thèmes(query='Relations nord-sud',save=True, print_=True, topic_max=50, revue='ei',corpus = Corpus_fouille_EI, dictionary = dictionary_EI_2)

**RI**

In [ ]:
for theme in champ_lexical_RI.keys():
    themes_lda = get_topics_lda(theme = theme, num_topics=10, thèmes_revue_cohérents=thèmes_revue_cohérents_RI)
    print(themes_lda,theme)

In [ ]:
panda = influence_thèmes(save=True, print_=True, topic_max=50, revue='ri',corpus = Corpus_fouille_RI, dictionary = dictionary_RI_2)

In [ ]:
panda = influence_thèmes(query='Droits de la personne',save=True, print_=True, topic_max=50, revue='ri',corpus = Corpus_fouille_RI, dictionary = dictionary_RI_2)

In [ ]:
panda = influence_thèmes(query='Temps de travail',save=True, print_=True, topic_max=50, revue='ri',corpus = Corpus_fouille_RI, dictionary = dictionary_RI_2)

# Calcul du ${\alpha}$ de Krippendorff

## BUT

- Calcul du alpha de Krippendorff pour valider la fidélité interjuges

In [ ]:
import krippendorff
import numpy as np

In [ ]:
data_AE = ("3	2	3	4	1	3	3	1	2	4	4	1	2	3	1	5	4	2",
"4	2	1	2	4	3	2	3	1	4	1	3	4	1	1	3	1	1",
"3	3	3	4	3	3	5	1	2	5	4	1	3	4	1	5	2	1",
"3	3	4	3	2	3	4	1	3	3	3	1	2	4	2	4	3	2",
"3	3	4	3	2	3	3	2	3	3	4	2	3	4	2	4	4	2",
"3	2	2	4	1	2	3	1	3	3	3	1	2	4	2	5	5	1",
"3	3	3	5	2	2	4	1	3	3	4	2	2	4	1	3	4	2",
"3	3	3	3	2	2	3	1	3	3	3	2	3	2	3	3	3	3",
"3	2	4	3	1	2	3	1	3	4	2	1	4	5	1	4	4	2",
"3	2	3	3	2	2	4	2	4	3	4	1	3	4	1	3	4	2",
"4	3	4	5	2	2	4	1	3	5	5	2	3	5	2	5	5	2",
"2	3	2	5	1	2	3	1	3	3	4	1	4	2	2	5	3	1",
"4	4	4	4	3	3	3	2	3	4	4	2	3	3	2	4	4	4",
"4	2	2	2	1	2	4	1	2	3	2	1	2	4	1	4	3	2",
"3	3	3	3	3	2	3	2	3	3	3	3	2	3	2	3	3	2")

In [ ]:
data_EI = ("1	1	1	1	1	1	1	1	1	1	1	2	4	4	4	3	2	3",
"2	1	1	1	1	2	1	1	1	4	3	3	5	5	5	2	2	3",
"2	1	2	3	1	1	3	1	2	4	4	5	5	5	4	4	5	4",
"1	1	1	1	1	1	1	1	1	2	2	2	2	3	2	1	2	1",
"2	1	2	2	2	1	2	1	2	3	3	3	3	4	3	3	3	3",
"2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2",
"2	1	1	2	1	1	1	1	1	1	3	4	4	3	5	3	4	3")

In [ ]:
data = ("2  1  0","1  2  3")
reliability_data = [[np.nan if v == '*' else int(v) for v in coder.split()] for coder in data]
krippendorff.alpha(reliability_data)

In [ ]:
data_RI = ("2	1	1	1	1	1	1	1	1	3	2	4	4	3	3	4	3	5",
"1	2	2	1	1	2	1	1	1	2	5	3	3	5	2	4	2	4",
"2	1	3	1	1	1	1	1	1	5	4	4	3	5	3	5	1	3",
"2	1	1	1	1	2	1	1	2	4	2	5	5	4	4	5	3	5",
"1	1	1	1	1	1	1	1	1	1	2	2	3	3	2	3	1	2",
"1	1	2	1	1	1	1	1	1	1	3	3	2	4	3	3	1	3",
"1	2	3	1	1	2	2	1	2	3	3	4	4	4	3	5	2	5",
"1	1	1	1	1	1	2	1	1	2	2	2	3	4	2	4	2	2",
"4	2	4	2	2	4	1	2	3	5	1	4	5	3	3	4	2	4",
"2	1	2	1	1	2	1	1	2	2	4	3	3	3	3	4	2	4",
"1	1	2	1	1	1	1	1	1	2	2	3	2	3	3	3	2	4",
"1	2	2	2	1	2	1	1	1	4	3	4	4	3	4	4	3	4")

In [ ]:
len(data_RI)

In [ ]:
reliability_data_AE = [[np.nan if v == '*' else int(v) for v in coder.split()] for coder in data_AE]
reliability_data_EI = [[np.nan if v == '*' else int(v) for v in coder.split()] for coder in data_EI]
reliability_data_RI = [[np.nan if v == '*' else int(v) for v in coder.split()] for coder in data_RI]

In [ ]:
print("Alpha AE : ", krippendorff.alpha(reliability_data_AE),
"Alpha EI : ", krippendorff.alpha(reliability_data_EI),
"Alpha RI : ", krippendorff.alpha(reliability_data_RI))